In [1]:
!pip install transformers torch sentence-transformers faiss-cpu wikipedia datetime pytz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 32.7 MB/s eta 0:00:00
   

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipedia
from datetime import datetime
import pytz
import random
import re
import warnings

warnings.filterwarnings('ignore')

class NexTalkBot:
    def __init__(self):
        # Initialize Models
        self.compute_device = "cuda" if torch.cuda.is_available() else "cpu"
        self.dialogue_model_name = "microsoft/DialoGPT-large"
        self.nlp_tokenizer = AutoTokenizer.from_pretrained(self.dialogue_model_name)
        self.nlp_model = AutoModelForCausalLM.from_pretrained(self.dialogue_model_name).to(self.compute_device)
        self.response_pipeline = pipeline(
            "text-generation",
            model=self.nlp_model,
            tokenizer=self.nlp_tokenizer,
            device=0 if self.compute_device == "cuda" else -1,
        )

        # Semantic Retriever
        self.semantic_encoder = SentenceTransformer('all-MiniLM-L6-v2').to(self.compute_device)
        self.vector_index = faiss.IndexFlatL2(384)

        # Conversation Context
        self.chat_log = []
        self.history_window = 6

        # Bot Identity
        self.assistant_name = "NexTalk"
        self.client_name = "User"
        self.local_timezone = pytz.timezone('UTC')

        # Built-in Skills
        self.utility_skills = {
            "get_time": self.fetch_time,
            "get_date": self.fetch_date,
            "weather_info": self.provide_weather_update,
            "tell_joke": self.share_joke,
            "do_math": self.evaluate_expression,
            "wiki_lookup": self.wikipedia_info,
        }

        # Greeting Message
        self.intro_message = f"""
        ✨ **Welcome to {self.assistant_name} - Your Next-Gen AI Companion!** ✨
        - Chat freely (ask about anything: jokes, time, math, info)
        - Type `/help` to view available commands
        - Type `quit` to exit
        Current System Time: {self.fetch_time()}
        """

    # === Utility Feature Methods ===
    def fetch_time(self):
        return datetime.now(self.local_timezone).strftime("%H:%M:%S")

    def fetch_date(self):
        return datetime.now(self.local_timezone).strftime("%Y-%m-%d")

    def provide_weather_update(self):
        weather_comments = [
            "I'm not connected to live weather feeds, but it’s sunny in my cloud! ☀️",
            "Forecast in my servers: 100% uptime and clear skies!",
            "Weather? I wish I had a window... Hope it's nice outside for you!"
        ]
        return random.choice(weather_comments)

    def share_joke(self):
        funny_lines = [
            "Why don't skeletons fight each other? They don't have the guts!",
            "How does a penguin build its house? Igloos it together!",
            "Why did the math book look sad? Because it had too many problems!"
        ]
        return random.choice(funny_lines)

    def evaluate_expression(self, question):
        try:
            sanitized = re.sub(r'[^0-9+\-*/(). ]', '', question)
            return f"🧮 Computed Result: {eval(sanitized)}"
        except:
            return "Hmm, that didn't work. Try a simple math like '12 / 4'."

    def wikipedia_info(self, search_query):
        try:
            result = wikipedia.summary(search_query, sentences=2)
            return f"📖 Wiki says: {result}"
        except:
            return "Sorry, I couldn't find relevant information on Wikipedia."

    # === Core Chatbot Logic ===
    def knowledge_lookup(self, user_text):
        """Skill based response handler"""
        user_text_lower = user_text.lower()
        if "your name" in user_text_lower or "who are you" in user_text_lower:
          return f"My name is {self.assistant_name}! 🤖"
        if "time" in user_text_lower:
          return f"⏳ Right now it is {self.fetch_time()}"
        elif "date" in user_text_lower:
          return f"📅 Today's date: {self.fetch_date()}"
        elif "weather" in user_text_lower:
          return self.provide_weather_update()
        elif "joke" in user_text_lower:
          return self.share_joke()
        elif any(op in user_text for op in ['+', '-', '*', '/']):
          return self.evaluate_expression(user_text)
        elif "who is" in user_text_lower or "what is" in user_text_lower:
          keywords = user_text.split("is")[-1].strip()
          return self.wikipedia_info(keywords)

        return None

    def create_response(self, query_input):
        """Handles user input and returns AI response"""
        if query_input.lower() in ['quit', 'exit', 'bye']:
            return "👋 Goodbye! Come back whenever you want to chat!"

        if query_input.lower() == '/help':
            return (
                "🛠️ **Available Commands:**\n"
                "- General questions\n"
                "- `time` → Ask for current time\n"
                "- `date` → Ask for today's date\n"
                "- `weather` → Get a weather comment\n"
                "- `joke` → Hear a funny joke\n"
                "- `calculate <expression>` → Simple math (like `7*9`)\n"
                "- `who is <someone>` or `what is <something>` → Wiki summary\n"
                "- `your name` → Know me better!\n"
            )

        # Skill Handler
        skill_response = self.knowledge_lookup(query_input)
        if skill_response:
            return skill_response

        # Generate Response using Dialogue Model
        past_conversation = (
            f"{' '.join(self.chat_log[-self.history_window:])}\n"
            f"{self.client_name}: {query_input}\n"
            f"{self.assistant_name}:"
        )

        result = self.response_pipeline(
            past_conversation,
            max_length=150,
            num_return_sequences=1,
            temperature=0.8,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.15,
            do_sample=True,
        )

        ai_reply = result[0]['generated_text'].split(f"{self.assistant_name}:")[-1].strip()
        ai_reply = re.sub(r'<\|.*?\|>', '', ai_reply)

        if ai_reply.lower() == query_input.lower():
            return "🤔 That's interesting! Could you tell me more?"

        return ai_reply

    def initiate_chat(self):
        print(self.intro_message)

        while True:
            try:
                user_text = input(f"{self.client_name}: ").strip()
                if not user_text:
                    continue

                bot_reply = self.create_response(user_text)
                print(f"{self.assistant_name}: {bot_reply}")

                self.chat_log.extend([user_text, bot_reply])
                if len(self.chat_log) > self.history_window * 2:
                    self.chat_log = self.chat_log[-self.history_window * 2:]

                if user_text.lower() in ['quit', 'exit', 'bye']:
                    break

            except KeyboardInterrupt:
                print(f"\n{self.assistant_name}: ✨ Until next time!")
                break
            except Exception as err:
                print(f"{self.assistant_name}: Oops! Something went wrong. Let's retry.")
                continue



In [4]:
# Initialize & run
nex_assistant = NexTalkBot()
nex_assistant.initiate_chat()

Device set to use cpu



        ✨ **Welcome to NexTalk - Your Next-Gen AI Companion!** ✨
        - Chat freely (ask about anything: jokes, time, math, info)
        - Type `/help` to view available commands
        - Type `quit` to exit
        Current System Time: 08:02:13
        
User: What is your name ?
NexTalk: My name is NexTalk! 🤖
User: what date today is?
NexTalk: 📅 Today's date: 2025-05-10
User: Hi, what's the weather like?
NexTalk: Forecast in my servers: 100% uptime and clear skies!
User: who is Narendra Modi
NexTalk: 📖 Wiki says: Narendra Damodardas Modi (born 17 September 1950) is an Indian politician who has served as the  prime minister of India since 2014. Modi was the chief minister of Gujarat from 2001 to 2014 and is the member of parliament (MP) for Varanasi.
User: quit
NexTalk: 👋 Goodbye! Come back whenever you want to chat!
